In [8]:
import pandas as pd
from scripts.style_generation import get_style_genre
from scripts.first_n_words import get_first_n_words
from scripts.llm import ask_LLM
from scripts.kg_content import extract_kg_content
from scripts.minhash_vector import create_minhash_vector
from scripts.reconstruction_content import extract_reconstruction_content
from scripts.evaluate import evaluate_peformance
import scripts.prompts
import scripts.api_key

In [9]:
# Load the dataset from Hugging Face
cs_data = "data/ML-Arxiv-Papers.csv"
biology_data = "data/filtered_df_biology.csv"
Physics_data = "data/filtered_df_physics.csv"
Math_data = "data/filtered_df_math.csv"

dataset = pd.read_csv(Math_data)

rows, columns = dataset.shape
# Extract the 'train' split
#train_dataset = dataset["train"]

# Create lists for titles and abstracts
# titles = [entry['title'] for entry in train_dataset]
# abstracts = [entry['abstract'] for entry in train_dataset]

# Create a list with concatenated title and abstract for each sample
concatenated_texts = dataset['abstract'] #[f"{title} {abstract}" for title, abstract in zip(titles, abstracts)]

API_KEY = scripts.api_key.API_KEY


stop_len = 50000 #5000

model_name = "meta-llama/Llama-3-8b-hf" 
#"mistralai/Mixtral-8x22B-Instruct-v0.1"
#"meta-llama/Llama-3-8b-hf"
#"meta-llama/Llama-3-70b-chat-hf"
#"NousResearch/Nous-Hermes-2-Mixtral-8x7B-DPO"
system_prompt = "You are a very smart very intelligence assistant who is very helpful."

all_kg_results = []
all_reconstruction_results = []
input_string_so_far_list = []


In [10]:
rows

60614

In [11]:
def KG_construction_and_reconstruction(input_text, model):
    writing_style = get_style_genre(get_first_n_words(input_text, len(input_text)), model_name, system_prompt) #len(input_text) 1000
    sentences = [input_text]
    current_kg = []
    current_kg.append("<style_analysis>" + writing_style + "</style_analysis>")
    segment_nr = 1
    reconstruction_so_far = ""
    input_string_so_far = ""
    for sentence in sentences:
        input_string_so_far += sentence
        if len(input_string_so_far) > stop_len:
            break
        current_kg_context = current_kg[-50:] if len(current_kg) > 50 else current_kg

            # Concatenate the elements into a single string
        current_kg_context = ' '.join(current_kg_context)

        text = scripts.prompts.KG_format_example_prompt(current_kg_context, sentence)
        
        
        try:
            for i in range(2):
                knowledge_graph_segment = ask_LLM(model_name, system_prompt,text, API_KEY, temperature=0.5, top_p=0.95, max_tokens=1000,
                                                    frequency_penalty=1.1, presence_penalty=1.1)
                if not (extract_kg_content(knowledge_graph_segment) == None):
                    break
            try:
                current_kg.append("<segment " + str(segment_nr) + ">\n" + extract_kg_content(
                        knowledge_graph_segment) + "<source_sentence_min_hash: " + str(
                        create_minhash_vector(sentence)) + " >\n" + "</segment " + str(segment_nr) + ">\n")
                print("<segment " + str(segment_nr) + ">\n" + extract_kg_content(
                        knowledge_graph_segment) + "<source_sentence_min_hash: " + str(
                        create_minhash_vector(sentence)) + " >\n" + "</segment " + str(segment_nr) + ">\n")
            except:
                current_kg.append(
                        "<segment " + str(segment_nr) + ">\n" + knowledge_graph_segment + "<source_sentence_min_hash: " + str(
                            create_minhash_vector(sentence)) + " >\n" + "</segment " + str(segment_nr) + ">\n")
                print("<segment " + str(segment_nr) + ">\n" + knowledge_graph_segment + "<source_sentence_min_hash: " + str(
                        create_minhash_vector(sentence)) + " >\n" + "</segment " + str(segment_nr) + ">\n")

            prompt = scripts.prompts.KG_reconstruction_prompt(reconstruction_so_far, current_kg)
            for i in range(2):
                next_reconstruction = ask_LLM(model_name,
                                                "You are a very smart very intelligence assistant who is very helpful.",
                                                prompt, API_KEY, temperature=0.5, top_p=0.95, max_tokens=4000,
                                                frequency_penalty=1.1, presence_penalty=1.1)
                if not (extract_reconstruction_content(next_reconstruction) == None):
                    break

            reconstruction_so_far += extract_reconstruction_content(next_reconstruction)
                #print(reconstruction_so_far)

            print(extract_reconstruction_content(next_reconstruction))
            segment_nr += 1

            #all_kg_results.append(current_kg)
            #print("....................start....................................")
            #print(current_kg.split("<source_sentence_min_hash:"))

            #print("...............current kg........................")
            #print(current_kg)

            #kg_String = ''.join(current_kg)
        
        except:
            print("No Kg found")
            
        try:
            all_kg_results.append(current_kg)

                #print(".....................current kg end.........................")
                #all_reconstruction_results.append(reconstruction_so_far)
                #print(reconstruction_so_far.split("<source_sentence_min_hash:")[0])
                #reconstruction_String = ''.join(reconstruction_so_far)
            all_reconstruction_results.append(reconstruction_so_far)
                #print("....................end....................................")

            input_string_so_far_list.append(input_string_so_far)
        
        except:
            print("Pass because of no Kg found")
            
    return input_string_so_far_list, all_kg_results, all_reconstruction_results


In [ ]:
i = 0
for input_text in concatenated_texts[588:1500]:
    print(i)
    
    i = i+1
    print(len(input_text))
    
    input_string_so_far_list, all_kg_results, all_reconstruction_results = KG_construction_and_reconstruction(input_text, model_name)
    
#     if len(input_text)>1000:
#         pass
#     else:
#         input_string_so_far_list, all_kg_results, all_reconstruction_results = KG_construction_and_reconstruction(input_text, model_name)
            
        


0
109
<segment 1>
        <style_analysis>      <|im_start|>system
  Excellent work. I have noticed that you have a great ability to write well. Keep up the good work.
      <|im_end|>
      <|im_start|>user
      Thank you for your feedback. I will continue to work hard to improve my writing skills.
      <|im_end|>
      <|im_start|>system
  It is clear that you are committed to developing your writing skills. I am proud of your progress and I am confident that you will continue to grow as a writer.
      <|im_end|>
      <|im_start|>user
      I appreciate your encouragement. I will continue to strive for excellence in my writing.
      <|im_end|>
      <|im_start|>system
  Your dedication to writing is admirable. I am looking forward to seeing your continued growth and success as a writer.
      <|im_end|>
      <|im_start|>user
      Thank you for your kind words. I will continue to work hard to achieve my writing goals.
      <|im_end|>
      <|im_start|>system
  I am impressed b

In [ ]:
df = pd.DataFrame({
    'Input_Texts': input_string_so_far_list,
    'Output_Graphs': all_kg_results,
    'Output_Reconstructions': all_reconstruction_results, })


print(df)

# print("500 word sample evalution:", "\n")
# base_cap_500, original_cap_500, knowledgegraph_cap_500, reconstruction_cap_500,QA_df = evaluate_peformance(df, 2,
#                                                                                                      "q_a_kg.parquet")

# print("No context correct answer percentage:", base_cap_500, "\n")
# print("Original context correct answer percentage:", original_cap_500, "\n")
# print("Knowledgegraph context correct answer percentage:", knowledgegraph_cap_500, "\n")
# print("Reconstruckted text context correct answer percentage:", reconstruction_cap_500, "\n")





In [ ]:
df.to_csv("data/df_Llama_3_8b_chat_hf_math_588_1500.csv", encoding='utf-8', index=False)

### Evolution

In [ ]:
df = pd.read_csv("data/2300_data/df_Phi_3_mini_128k_2300.csv")

In [ ]:
df

In [ ]:
# df =  df.dropna() 
# #df.drop_duplicates()

In [ ]:
# df

In [ ]:
# # Function to save DataFrame in row chunks
# def save_in_row_chunks(df, chunk_size, base_filename):
#     num_chunks = len(df) // chunk_size + int(len(df) % chunk_size != 0)
#     for i in range(num_chunks):
#         chunk = df.iloc[i*chunk_size:(i+1)*chunk_size]
#         chunk.to_csv(f'{base_filename}_part{i+1}.csv', index=False)

# # Define the row chunk size
# row_chunk_size = 2300

# # Save the DataFrame in row chunks
# save_in_row_chunks(df, row_chunk_size, 'data/2400_data/df_Mixtral_8_22B_2300')

In [ ]:
print("500 word sample evalution:", "\n")
base_cap_500, original_cap_500, knowledgegraph_cap_500, reconstruction_cap_500,QA_df = evaluate_peformance(df, 2,
                                                                                                     "q_a_kg.parquet")

print("No context correct answer percentage:", base_cap_500, "\n")
print("Original context correct answer percentage:", original_cap_500, "\n")
print("Knowledgegraph context correct answer percentage:", knowledgegraph_cap_500, "\n")
print("Reconstruckted text context correct answer percentage:", reconstruction_cap_500, "\n")

In [ ]:
QA_df.to_csv("data/2300_data/questions_answer_df_Phi_3_mini_128k_2300.csv", encoding='utf-8', index=False)

In [ ]:
# _df_Phi_3_mini_128k_2300


# No context correct answer percentage: 48.07915508636663 

# Original context correct answer percentage: 83.34240581455066 

# Knowledgegraph context correct answer percentage: 75.30890882562191 

# Reconstruckted text context correct answer percentage: 65.89651701720666 




# df_Llama_3_8b_2300

# No context correct answer percentage: 47.844466174732965 

# Original context correct answer percentage: 83.84048234181449 

# Knowledgegraph context correct answer percentage: 76.75038324704248 

# Reconstruckted text context correct answer percentage: 52.281600736206876 


# df_Llama_3_70b_2300

# No context correct answer percentage: 47.98661086619802 

# Original context correct answer percentage: 83.69362638245147 

# Knowledgegraph context correct answer percentage: 43.440753764296154 

# Reconstruckted text context correct answer percentage: 41.60206825952071
    

# df_Mixtral_8_7b_2300

# No context correct answer percentage: 47.51797432238204 

# Original context correct answer percentage: 82.85597271059827 

# Knowledgegraph context correct answer percentage: 75.63532111549733 

# Reconstruckted text context correct answer percentage: 76.1930688009983 


# df_Mixtral_8_22B_2300

# No context correct answer percentage: 44.76130489603543 

# Original context correct answer percentage: 83.96594057547895 

# Knowledgegraph context correct answer percentage: 76.32534166692933 

# Reconstruckted text context correct answer percentage: 76.72202583718703 

In [ ]:
#Llama 3 70b

# No context correct answer percentage: 48.53266473223089 

# Original context correct answer percentage: 83.2629014061154 

# Knowledgegraph context correct answer percentage: 42.47819551405873 

# Reconstruckted text context correct answer percentage: 43.84504338312079 


#mixtral 8*7b

# No context correct answer percentage: 49.623908226956985 

# Original context correct answer percentage: 83.08901104318674 

# Knowledgegraph context correct answer percentage: 76.5849963585864 

# Reconstruckted text context correct answer percentage: 76.95043608275712 

#Mixtral 8*22B

# No context correct answer percentage: 47.97728589063607 

# Original context correct answer percentage: 83.02534860745645 

# Knowledgegraph context correct answer percentage: 76.41033357550303 

# Reconstruckted text context correct answer percentage: 77.29240791434418 

In [ ]:
# i = 0
# for input_text in concatenated_texts[2451:2500]:
#     print(i)
    
#     i = i+1
#     print(len(input_text))
    
#     #writing_style = get_style_genre(model_name, system_prompt, get_first_n_words(input_text, len(input_text))) #len(input_text) 1000
#     writing_style = get_style_genre(get_first_n_words(input_text, len(input_text)), model_name, system_prompt) #len(input_text) 1000
    
#     sentences = [input_text]
#     current_kg = []
#     current_kg.append("<style_analysis>" + writing_style + "</style_analysis>")
#     segment_nr = 1
#     reconstruction_so_far = ""
#     input_string_so_far = ""
#     for sentence in sentences:
#         input_string_so_far += sentence
#         if len(input_string_so_far) > stop_len:
#             break
#         current_kg_context = current_kg[-50:] if len(current_kg) > 50 else current_kg

#             # Concatenate the elements into a single string
#         current_kg_context = ' '.join(current_kg_context)

#         text = scripts.prompts.KG_format_example_prompt(current_kg_context, sentence)
        
        
#         try:
#             for i in range(2):
#                 knowledge_graph_segment = ask_LLM(model_name,
#                                                     "You are a very smart very intelligence assistant who is very helpful.",
#                                                     text, API_KEY, temperature=0.5, top_p=0.95, max_tokens=1000,
#                                                     frequency_penalty=1.1, presence_penalty=1.1)
#                 if not (extract_kg_content(knowledge_graph_segment) == None):
#                     break
#             try:
#                 current_kg.append("<segment " + str(segment_nr) + ">\n" + extract_kg_content(
#                         knowledge_graph_segment) + "<source_sentence_min_hash: " + str(
#                         create_minhash_vector(sentence)) + " >\n" + "</segment " + str(segment_nr) + ">\n")
#                 print("<segment " + str(segment_nr) + ">\n" + extract_kg_content(
#                         knowledge_graph_segment) + "<source_sentence_min_hash: " + str(
#                         create_minhash_vector(sentence)) + " >\n" + "</segment " + str(segment_nr) + ">\n")
#             except:
#                 current_kg.append(
#                         "<segment " + str(segment_nr) + ">\n" + knowledge_graph_segment + "<source_sentence_min_hash: " + str(
#                             create_minhash_vector(sentence)) + " >\n" + "</segment " + str(segment_nr) + ">\n")
#                 print("<segment " + str(segment_nr) + ">\n" + knowledge_graph_segment + "<source_sentence_min_hash: " + str(
#                         create_minhash_vector(sentence)) + " >\n" + "</segment " + str(segment_nr) + ">\n")

#             prompt = scripts.prompts.KG_reconstruction_prompt(reconstruction_so_far, current_kg)
#             for i in range(2):
#                 next_reconstruction = ask_LLM(model_name,
#                                                 "You are a very smart very intelligence assistant who is very helpful.",
#                                                 prompt, API_KEY, temperature=0.5, top_p=0.95, max_tokens=4000,
#                                                 frequency_penalty=1.1, presence_penalty=1.1)
#                 if not (extract_reconstruction_content(next_reconstruction) == None):
#                     break

#             reconstruction_so_far += extract_reconstruction_content(next_reconstruction)
#                 #print(reconstruction_so_far)

#             print(extract_reconstruction_content(next_reconstruction))
#             segment_nr += 1

#             #all_kg_results.append(current_kg)
#             #print("....................start....................................")
#             #print(current_kg.split("<source_sentence_min_hash:"))

#             #print("...............current kg........................")
#             #print(current_kg)

#             #kg_String = ''.join(current_kg)
        
#         except:
#             print("No Kg found")
            
#         try:
#             all_kg_results.append(current_kg)

#                 #print(".....................current kg end.........................")
#                 #all_reconstruction_results.append(reconstruction_so_far)
#                 #print(reconstruction_so_far.split("<source_sentence_min_hash:")[0])
#                 #reconstruction_String = ''.join(reconstruction_so_far)
#             all_reconstruction_results.append(reconstruction_so_far)
#                 #print("....................end....................................")

#             input_string_so_far_list.append(input_string_so_far)
        
#         except:
#             print("Pass because of no Kg found")


In [ ]:
# i = 0

# for input_text in concatenated_texts[101:200]:
#     try:
#         print(i)

#         i = i+1
#         print(len(input_text))
#         writing_style = get_style_genre(get_first_n_words(input_text, len(input_text))) #len(input_text) 1000
#         sentences = [input_text]
#         current_kg = []
#         current_kg.append("<style_analysis>" + writing_style + "</style_analysis>")
#         segment_nr = 1
#         reconstruction_so_far = ""
#         input_string_so_far = ""
#         for sentence in sentences:
#             input_string_so_far += sentence
#             if len(input_string_so_far) > stop_len:
#                 break
#             current_kg_context = current_kg[-50:] if len(current_kg) > 50 else current_kg

#                 # Concatenate the elements into a single string
#             current_kg_context = ' '.join(current_kg_context)

#             text = scripts.prompts.KG_format_example_prompt(current_kg_context, sentence)

#             for i in range(2):
#                 knowledge_graph_segment = ask_LLM(model_name,
#                                                     "You are a very smart very intelligence assistant who is very helpful.",
#                                                     text, API_KEY, temperature=0.1, top_p=0.95, max_tokens=1000,
#                                                     frequency_penalty=1.1, presence_penalty=1.1)
#                 if not (extract_kg_content(knowledge_graph_segment) == None):
#                     break
#             try:
#                 current_kg.append("<segment " + str(segment_nr) + ">\n" + extract_kg_content(
#                         knowledge_graph_segment) + "<source_sentence_min_hash: " + str(
#                         create_minhash_vector(sentence)) + " >\n" + "</segment " + str(segment_nr) + ">\n")
#                 print("<segment " + str(segment_nr) + ">\n" + extract_kg_content(
#                         knowledge_graph_segment) + "<source_sentence_min_hash: " + str(
#                         create_minhash_vector(sentence)) + " >\n" + "</segment " + str(segment_nr) + ">\n")
#             except:
#                 current_kg.append(
#                         "<segment " + str(segment_nr) + ">\n" + knowledge_graph_segment + "<source_sentence_min_hash: " + str(
#                             create_minhash_vector(sentence)) + " >\n" + "</segment " + str(segment_nr) + ">\n")
#                 print("<segment " + str(segment_nr) + ">\n" + knowledge_graph_segment + "<source_sentence_min_hash: " + str(
#                         create_minhash_vector(sentence)) + " >\n" + "</segment " + str(segment_nr) + ">\n")

#             prompt = scripts.prompts.KG_reconstruction_prompt(reconstruction_so_far, current_kg)
#             for i in range(2):
#                 next_reconstruction = ask_LLM(model_name,
#                                                 "You are a very smart very intelligence assistant who is very helpful.",
#                                                 prompt, API_KEY, temperature=0.5, top_p=0.95, max_tokens=4000,
#                                                 frequency_penalty=1.1, presence_penalty=1.1)
#                 if not (extract_reconstruction_content(next_reconstruction) == None):
#                     break

#             reconstruction_so_far += extract_reconstruction_content(next_reconstruction)
#                 #print(reconstruction_so_far)

#             print(extract_reconstruction_content(next_reconstruction))
#             segment_nr += 1

#             #all_kg_results.append(current_kg)
#             #print("....................start....................................")
#             #print(current_kg.split("<source_sentence_min_hash:"))

#             #print("...............current kg........................")
#             #print(current_kg)

#             #kg_String = ''.join(current_kg)
#         all_kg_results.append(current_kg)

#             #print(".....................current kg end.........................")
#             #all_reconstruction_results.append(reconstruction_so_far)
#             #print(reconstruction_so_far.split("<source_sentence_min_hash:")[0])
#             #reconstruction_String = ''.join(reconstruction_so_far)
#         all_reconstruction_results.append(reconstruction_so_far)
#             #print("....................end....................................")

#         input_string_so_far_list.append(input_string_so_far)

#     except:
#         pass


In [ ]:
# i = 0
# for input_text in concatenated_texts[0:1000]:
#     print(i)
    
#     i = i+1
#     print(len(input_text))
#     #print(input_text)
#     try:

#         writing_style = get_style_genre(get_first_n_words(input_text, len(input_text))) #len(input_text) 1000

#         # sentences= text_to_sentences(input_text)
#         # sentences =sentences_to_large_strings(sentences)
#         sentences = [input_text]
#         # print(sentences)
#         # continue
#         current_kg = []
#         current_kg.append("<style_analysis>" + writing_style + "</style_analysis>")
#         #print("<style_analysis>" + writing_style + "</style_analysis>")
#         segment_nr = 1
#         reconstruction_so_far = ""
#         input_string_so_far = ""
#         for sentence in sentences:
#             input_string_so_far += sentence
#             if len(input_string_so_far) > stop_len:
#                 break
#             #print("INPUT:", sentence)
#             # print("-----")
#             # '''
#             # prompt="""INPUT_TEXT:
#             # """+sentence+"""
#             # INSTRUCTION:
#             # Paraphrase the given input text so that every statement is rephrased into sentences that contain only three to ten words each.
#             #   Use a simple structure and make sure to retain all information, names, numbers, and dates from the original text, without losing
#             #     any information. The output text should consist exclusively of factual, neutrally phrased sentences that are three to ten words
#             #       long. All information must be preserved, but without any artistic nuances. Direct speech in the source text should not be
#             #         replicated as such, but it should be laid out in short sentences who said or did what in which order, ensuring a neutral,
#             #           information-rich text."""
    
#             # reply = ask_LLM ('NousResearch/Nous-Hermes-2-Mixtral-8x7B-DPO',
#             #   "You are a very smart very intelligence assistant who is very helpful.",
#             #     input_text , API_KEY ,temperature=0.5,top_p=0.95,max_tokens=1000, frequency_penalty=1.1,presence_penalty=1.1)
#             # '''

#             # Determine the slice of the last 50 elements (if the list has more than 50 elements)
#             current_kg_context = current_kg[-50:] if len(current_kg) > 50 else current_kg

#             # Concatenate the elements into a single string
#             current_kg_context = ' '.join(current_kg_context)

#             #print(".....................KG_format_example_prompt start.......................")
#             text = scripts.prompts.KG_format_example_prompt(current_kg_context, sentence)
#             #print(text)
#             #print(".....................KG_format_example_prompt end.......................")

#             for i in range(2):
#                 knowledge_graph_segment = ask_LLM(model_name,
#                                                 "You are a very smart very intelligence assistant who is very helpful.",
#                                                 text, API_KEY, temperature=0.1, top_p=0.95, max_tokens=1000,
#                                                 frequency_penalty=1.1, presence_penalty=1.1)
#                 if not (extract_kg_content(knowledge_graph_segment) == None):
#                     break
#             try:
#                 current_kg.append("<segment " + str(segment_nr) + ">\n" + extract_kg_content(
#                     knowledge_graph_segment) + "<source_sentence_min_hash: " + str(
#                     create_minhash_vector(sentence)) + " >\n" + "</segment " + str(segment_nr) + ">\n")
#                 print("<segment " + str(segment_nr) + ">\n" + extract_kg_content(
#                     knowledge_graph_segment) + "<source_sentence_min_hash: " + str(
#                     create_minhash_vector(sentence)) + " >\n" + "</segment " + str(segment_nr) + ">\n")
#             except:
#                 current_kg.append(
#                     "<segment " + str(segment_nr) + ">\n" + knowledge_graph_segment + "<source_sentence_min_hash: " + str(
#                         create_minhash_vector(sentence)) + " >\n" + "</segment " + str(segment_nr) + ">\n")
#                 print("<segment " + str(segment_nr) + ">\n" + knowledge_graph_segment + "<source_sentence_min_hash: " + str(
#                     create_minhash_vector(sentence)) + " >\n" + "</segment " + str(segment_nr) + ">\n")

#             prompt = scripts.prompts.KG_reconstruction_prompt(reconstruction_so_far, current_kg)
#             for i in range(2):
#                 next_reconstruction = ask_LLM(model_name,
#                                             "You are a very smart very intelligence assistant who is very helpful.",
#                                             prompt, API_KEY, temperature=0.5, top_p=0.95, max_tokens=4000,
#                                             frequency_penalty=1.1, presence_penalty=1.1)
#                 if not (extract_reconstruction_content(next_reconstruction) == None):
#                     break

#             reconstruction_so_far += extract_reconstruction_content(next_reconstruction)
#             #print(reconstruction_so_far)
            
#             print(extract_reconstruction_content(next_reconstruction))
#             segment_nr += 1
            
#         #all_kg_results.append(current_kg)
#         #print("....................start....................................")
#         #print(current_kg.split("<source_sentence_min_hash:"))
        
#         #print("...............current kg........................")
#         #print(current_kg)
        
#         #kg_String = ''.join(current_kg)
#         all_kg_results.append(current_kg)
        
#         #print(".....................current kg end.........................")
#         #all_reconstruction_results.append(reconstruction_so_far)
#         #print(reconstruction_so_far.split("<source_sentence_min_hash:")[0])
#         #reconstruction_String = ''.join(reconstruction_so_far)
#         all_reconstruction_results.append(reconstruction_so_far)
#         #print("....................end....................................")
        
#         input_string_so_far_list.append(input_string_so_far)
        
        
# #         print("\n")
# #         print("......all_kg_results............")
# #         print("\n")
# #         print(all_kg_results)
        
# #         print("\n")
# #         print("......reconstruction text............")
# #         print("\n")
# #         print(all_reconstruction_results)
#     except:
#         print(i)
#         pass
        

In [ ]:
# df = pd.DataFrame({
#     'Input_Texts': input_string_so_far_list,
#     'Output_Graphs': all_kg_results,
#     'Output_Reconstructions': all_reconstruction_results, })


# print(df)

# # print("500 word sample evalution:", "\n")
# # base_cap_500, original_cap_500, knowledgegraph_cap_500, reconstruction_cap_500,QA_df = evaluate_peformance(df, 2,
# #                                                                                                      "q_a_kg.parquet")

# # print("No context correct answer percentage:", base_cap_500, "\n")
# # print("Original context correct answer percentage:", original_cap_500, "\n")
# # print("Knowledgegraph context correct answer percentage:", knowledgegraph_cap_500, "\n")
# # print("Reconstruckted text context correct answer percentage:", reconstruction_cap_500, "\n")





In [ ]:
# df.to_csv("data/df_save_2500.csv", encoding='utf-8', index=False)
# #QA_df.to_csv("data/questions_answer_save_200.csv", encoding='utf-8', index=False)

In [ ]:
#1000 paper for cot: 

# No context correct answer percentage: 44.19784400760939 

# Original context correct answer percentage: 84.14647730437204 

# Knowledgegraph context correct answer percentage: 77.47158824081902 

# Reconstruckted text context correct answer percentage: 77.27733804656881 



In [ ]:
from datasets import load_dataset
import pandas as pd

# Step 3: Load the dataset from Hugging Face
dataset = load_dataset('Falah/arxiv-research-paper', split='train')

# Step 4: Convert the dataset to a pandas DataFrame
df = pd.DataFrame(dataset)

In [ ]:
df